# Finding Similar Items: Textually Similar Documents

In [1]:
import findspark
findspark.init()

from pyspark import *
from pyspark.sql.functions import desc, col, rand
from pyspark.sql import *
from graphframes import *

import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import sys
from sympy.ntheory.generate import nextprime

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%env PYTHONHASHSEED 0

env: PYTHONHASHSEED=0


In [4]:
if os.environ.get("PYTHONHASHSEED") != "0":
    raise Exception("You must set PYTHONHASHSEED=0 when starting the Jupyter server to get reproducible results.")

In [5]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = sys.executable

In [6]:
spark = SparkSession.builder.appName('hw1').getOrCreate()

21/11/10 17:51:04 WARN Utils: Your hostname, mark-machine resolves to a loopback address: 127.0.1.1; using 192.168.1.141 instead (on interface wlp8s0)
21/11/10 17:51:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mark/.ivy2/cache
The jars for the packages stored in: /home/mark/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-af42575c-4fd5-479c-8363-d6e58f2f963d;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 107ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------

http://mlg.ucd.ie/datasets/bbc.html

In [7]:
%%bash
file=./data/bbc-fulltext.zip

if [ -e "$file" ]; then
    echo "$file exists, skipping"
else 
    echo "$file does not exist, using wget to download it"
    wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip -P data/
    unzip $file -d data/
fi

./data/bbc-fulltext.zip exists, skipping


In [8]:
def read_text(path):
    print(path)
    text = np.nan
    try:
        with open(path, 'r') as f:
            lines = f.readlines()

        lines = [line.strip() for line in lines]
        lines = list(filter(lambda x: x != "", lines))

        text = " ".join(lines)
    except Exception as e:
        print(f"skipping {path} due to: {e}")
    finally:
        return text

path = "data/bbc/sport/199.txt"
read_text(path)

data/bbc/sport/199.txt
skipping data/bbc/sport/199.txt due to: 'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte


nan

In [9]:
data_path = "data/bbc/"
topic_names = ["business", "entertainment", "politics", "sport", "tech"]

data = {}
paths = []
topics = []
texts = []

for topic_name in topic_names:
    dir_path = os.path.join(data_path, topic_name)
    
    for dirpath, dirnames, filenames in os.walk(dir_path):
        paths_ = [os.path.join(dirpath, filename) for filename in filenames]
        topics_ = [topic_name for i in range(len(paths_))]
        
        texts_ = [read_text(path_) for path_ in paths_]
        
        paths += paths_
        topics += topics_
        texts += texts_

data["path"] = paths
data["topic"] = topics
data["text"] = texts
data["id"] = [i for i in range(len(texts))]

pd_df = pd.DataFrame(data=data)
pd_df

data/bbc/business/489.txt
data/bbc/business/194.txt
data/bbc/business/461.txt
data/bbc/business/007.txt
data/bbc/business/145.txt
data/bbc/business/463.txt
data/bbc/business/294.txt
data/bbc/business/505.txt
data/bbc/business/348.txt
data/bbc/business/159.txt
data/bbc/business/123.txt
data/bbc/business/077.txt
data/bbc/business/002.txt
data/bbc/business/509.txt
data/bbc/business/370.txt
data/bbc/business/050.txt
data/bbc/business/072.txt
data/bbc/business/011.txt
data/bbc/business/314.txt
data/bbc/business/052.txt
data/bbc/business/355.txt
data/bbc/business/155.txt
data/bbc/business/412.txt
data/bbc/business/491.txt
data/bbc/business/105.txt
data/bbc/business/497.txt
data/bbc/business/061.txt
data/bbc/business/404.txt
data/bbc/business/125.txt
data/bbc/business/070.txt
data/bbc/business/477.txt
data/bbc/business/349.txt
data/bbc/business/237.txt
data/bbc/business/103.txt
data/bbc/business/167.txt
data/bbc/business/217.txt
data/bbc/business/211.txt
data/bbc/business/043.txt
data/bbc/bus

data/bbc/politics/298.txt
data/bbc/politics/380.txt
data/bbc/politics/334.txt
data/bbc/politics/359.txt
data/bbc/politics/084.txt
data/bbc/politics/138.txt
data/bbc/politics/049.txt
data/bbc/politics/180.txt
data/bbc/politics/216.txt
data/bbc/politics/158.txt
data/bbc/politics/273.txt
data/bbc/politics/168.txt
data/bbc/politics/333.txt
data/bbc/politics/367.txt
data/bbc/politics/170.txt
data/bbc/politics/141.txt
data/bbc/politics/414.txt
data/bbc/politics/078.txt
data/bbc/politics/199.txt
data/bbc/politics/076.txt
data/bbc/politics/181.txt
data/bbc/politics/089.txt
data/bbc/politics/128.txt
data/bbc/politics/378.txt
data/bbc/politics/006.txt
data/bbc/politics/413.txt
data/bbc/politics/252.txt
data/bbc/politics/362.txt
data/bbc/politics/223.txt
data/bbc/politics/282.txt
data/bbc/politics/356.txt
data/bbc/politics/133.txt
data/bbc/politics/376.txt
data/bbc/politics/175.txt
data/bbc/politics/274.txt
data/bbc/politics/120.txt
data/bbc/politics/323.txt
data/bbc/politics/297.txt
data/bbc/pol

data/bbc/politics/233.txt
data/bbc/politics/264.txt
data/bbc/politics/014.txt
data/bbc/politics/245.txt
data/bbc/politics/022.txt
data/bbc/politics/234.txt
data/bbc/politics/053.txt
data/bbc/politics/189.txt
data/bbc/politics/136.txt
data/bbc/politics/197.txt
data/bbc/politics/304.txt
data/bbc/politics/307.txt
data/bbc/politics/208.txt
data/bbc/politics/086.txt
data/bbc/politics/232.txt
data/bbc/politics/117.txt
data/bbc/politics/177.txt
data/bbc/politics/394.txt
data/bbc/politics/318.txt
data/bbc/sport/489.txt
data/bbc/sport/194.txt
data/bbc/sport/461.txt
data/bbc/sport/007.txt
data/bbc/sport/145.txt
data/bbc/sport/463.txt
data/bbc/sport/294.txt
data/bbc/sport/505.txt
data/bbc/sport/348.txt
data/bbc/sport/159.txt
data/bbc/sport/123.txt
data/bbc/sport/077.txt
data/bbc/sport/002.txt
data/bbc/sport/509.txt
data/bbc/sport/370.txt
data/bbc/sport/050.txt
data/bbc/sport/072.txt
data/bbc/sport/011.txt
data/bbc/sport/314.txt
data/bbc/sport/052.txt
data/bbc/sport/355.txt
data/bbc/sport/155.txt


,path,topic,text,id
0,data/bbc/business/489.txt,business,Criminal probe on Citigroup deals Traders at U...,0
1,data/bbc/business/194.txt,business,'Post-Christmas lull' in lending UK mortgage l...,1
2,data/bbc/business/461.txt,business,Beijingers fume over parking fees Choking traf...,2
3,data/bbc/business/007.txt,business,Jobs growth still slow in the US The US create...,3
4,data/bbc/business/145.txt,business,Winn-Dixie files for bankruptcy US supermarket...,4
...,...,...,...,...
2220,data/bbc/tech/232.txt,tech,BBC web search aids odd queries The BBC's onli...,2220
2221,data/bbc/tech/117.txt,tech,Joke e-mail virus tricks users A virus that di...,2221
2222,data/bbc/tech/177.txt,tech,Microsoft debuts security tools Microsoft is r...,2222
2223,data/bbc/tech/394.txt,tech,TV's future down the phone line Internet TV ha...,2223


In [10]:
pd_df.dropna(inplace=True)

In [11]:
df_data_all = spark.createDataFrame(pd_df, list(pd_df.columns.values))

In [12]:
df_data_all.show()

+--------------------+--------+--------------------+---+
|                path|   topic|                text| id|
+--------------------+--------+--------------------+---+
|data/bbc/business...|business|Criminal probe on...|  0|
|data/bbc/business...|business|'Post-Christmas l...|  1|
|data/bbc/business...|business|Beijingers fume o...|  2|
|data/bbc/business...|business|Jobs growth still...|  3|
|data/bbc/business...|business|Winn-Dixie files ...|  4|
|data/bbc/business...|business|US economy still ...|  5|
|data/bbc/business...|business|Wall Street cheer...|  6|
|data/bbc/business...|business|China now top tra...|  7|
|data/bbc/business...|business|US to probe airli...|  8|
|data/bbc/business...|business|India unveils ant...|  9|
|data/bbc/business...|business|Train strike grip...| 10|
|data/bbc/business...|business|China had role in...| 11|
|data/bbc/business...|business|Dollar gains on G...| 12|
|data/bbc/business...|business|Euro firms miss o...| 13|
|data/bbc/business...|business|

In [15]:
limit_to = 20
df_data = df_data_all.limit(limit_to)
df_data_collected = df_data.collect()
#data = data_all.orderBy(rand(seed=1234)).limit(limit_to)

In [16]:
print(f"data.shape = {df_data.count(), len(df_data.columns)}")

data.shape = (20, 4)


In [17]:
def shingles(text, k):
    shingles = list(set([text[i:i + k] for i in range(len(text) - k + 1)]))
    return shingles

def hash_shingle(shingle):
    return hash(shingle)

def hash_singles(shingles):
    return [hash_shingle(shingle) for shingle in shingles]

# transform to hashed shingles 
k = 10
myrdd = df_data.rdd.map(lambda x: shingles(x["text"], k=k))
myrdd2 = myrdd.map(lambda x: hash_singles(x))

# build shingle map
# for viz
myrdd3 = myrdd.flatMap(lambda x: x)
myrdd4 = myrdd3.map(lambda w: (w,1)).reduceByKey(lambda a, b: a+b)

# for use
myrdd5 = myrdd2.flatMap(lambda x: x)
myrdd6 = myrdd5.map(lambda w: (w,1)).reduceByKey(lambda a, b: a+b)
myrdd7 = myrdd6.map(lambda x: x[0])

In [18]:
myrdd.take(3)

[[' The move ',
  ' the world',
  'ch include',
  'st. "Germa',
  'e are disa',
  'ket manipu',
  'tion. Citi',
  ' £7.6bn) o',
  'red the in',
  'nds of the',
  'reported B',
  ' over a co',
  ' private b',
  'ies Tradin',
  'on to the ',
  ' it has to',
  's that if ',
  '($14.4bn; ',
  'aFin told ',
  'in Tokyo a',
  'te fully w',
  'e move was',
  're" led th',
  'n Citigrou',
  'tor the qu',
  'culture" l',
  ' the quest',
  'Fin told B',
  'nd evidenc',
  'he traders',
  'he public ',
  'ugh the ca',
  ' led the b',
  'ing rules.',
  ' in 2004. ',
  'has referr',
  't, with 4b',
  'ome Citigr',
  'd now tran',
  'in trades ',
  ' operation',
  's are foun',
  ' the publi',
  'in Germany',
  'The move w',
  'ly with th',
  'd the inve',
  '4bn; £7.6b',
  'roup are f',
  'lator has ',
  's of possi',
  'o the publ',
  'in Japan w',
  ' Trading A',
  'itigroup i',
  'nce, the U',
  'Securities',
  'ties Tradi',
  ' regulator',
  'ewhere in ',
  't would co',
  ' Japan was',
  'elf. 

In [19]:
myrdd2.take(2)

[[7111565423638994948,
  -5015560593384366068,
  -4369003706524983282,
  -1771218041006907373,
  7821535723265736728,
  8482422900521762855,
  5669648916202872882,
  -6958089004773203916,
  1399697435743895605,
  -1717999963076804540,
  8059833274335567965,
  -3383889895710760864,
  1256212927005950064,
  3953961759648301175,
  8985511375859908747,
  -166198657333600117,
  -7964618377302761330,
  3437529968616751262,
  -8406346974797661,
  -3650136105631227739,
  -4522937700181344087,
  5848866344305721519,
  -1326489434019561291,
  6514494971904295095,
  -2814141030687432520,
  -3449636775009075007,
  8884592349775896789,
  -354811029125316394,
  6976094793672966362,
  3029617905389265120,
  4684198829015392483,
  13281303806370019,
  852581877094605038,
  4421277232393437424,
  -8952170306448596746,
  -5033114232129634056,
  -6290613183430123269,
  -8701949344032440066,
  633572168750833921,
  1093486405499486466,
  -4132668428262743807,
  682851355854643460,
  -8467113678519238368,


In [20]:
myrdd3.sortBy(lambda x: x[1], ascending=True).collect()

['e are disa',
 's that if ',
 'e move was',
 'n Citigrou',
 'd now tran',
 's are foun',
 'd the inve',
 's of possi',
 'o the publ',
 't would co',
 's on 2 Aug',
 'a five-yea',
 ". Germany'",
 'y over a c',
 't back lat',
 'o the pros',
 'S banking ',
 'g operatio',
 'n Japan wa',
 '. "It is n',
 'n should b',
 'l law, pro',
 'd clues of',
 'f governme',
 'f the pros',
 'e on Citig',
 '. "Germany',
 'n the worl',
 'd trading ',
 'C News it ',
 'g giant Ci',
 's referred',
 '. Its priv',
 'e deal saw',
 't is now a',
 'e UK and e',
 'h the auth',
 '2 August. ',
 'n by regul',
 's of the p',
 'r an "aggr',
 'n said. "I',
 'r a contro',
 'd be broug',
 '" Citigrou',
 's to put t',
 ', under Ge',
 'd to the p',
 '" the spok',
 'e of possi',
 '. The move',
 'e of 11bn ',
 'c prosecut',
 'p itself. ',
 't had now ',
 't the BaFi',
 'r regulato',
 'd through ',
 'e a five-y',
 'f BaFin fi',
 'n regulato',
 'e to co-op',
 '. However,',
 'p deals Tr',
 's are stil',
 'e brought ',
 'd BBC Ne

In [21]:
myrdd4.sortBy(lambda x: x[1], ascending=False).collect()

[(' that the ', 10),
 (' governmen', 8),
 ('government', 8),
 (' However, ', 8),
 ('ording to ', 8),
 ('ccording t', 8),
 ('. Analysts', 8),
 (' Analysts ', 8),
 ('cording to', 8),
 (' economic ', 8),
 ('. However,', 7),
 ('overnment ', 7),
 (' said the ', 7),
 (' with the ', 6),
 (' last year', 6),
 (' increase ', 6),
 ('n December', 6),
 (' would be ', 6),
 ('more than ', 6),
 (' more than', 6),
 ('Analysts s', 6),
 ('gainst the', 6),
 (' against t', 6),
 ('against th', 6),
 ('ainst the ', 6),
 (' has been ', 6),
 ('n Thursday', 6),
 (' on Thursd', 6),
 ('ed by the ', 6),
 ('on Thursda', 6),
 (' companies', 6),
 (' continue ', 5),
 ('in Decembe', 5),
 (' in Decemb', 5),
 (' newspaper', 5),
 ('nalysts sa', 5),
 ('ankruptcy ', 5),
 ('bankruptcy', 5),
 (' bankruptc', 5),
 (' competiti', 5),
 (' slightly ', 5),
 (' spending ', 5),
 ('ast year, ', 5),
 ('uring the ', 5),
 ('companies ', 5),
 (' said it w', 4),
 ('d that the', 4),
 (' the same ', 4),
 (' January, ', 4),
 ('relatively', 4),

In [22]:
myrdd6.sortBy(lambda x: x[1], ascending=False).collect()

[(-2920061944228900013, 10),
 (-5832103434092574146, 8),
 (-2023258147501582425, 8),
 (-604390337244504599, 8),
 (3992740884027253113, 8),
 (7864054913084689232, 8),
 (-3596215347334417094, 8),
 (2825412531588765072, 8),
 (-510968863391319958, 8),
 (4550722251823365715, 8),
 (-1261740509138761883, 7),
 (5655413679353451606, 7),
 (-3393863091543464099, 7),
 (3263431647110166533, 6),
 (-3276136649668943504, 6),
 (7599208951150568023, 6),
 (8482201148957801853, 6),
 (-5617763565677040406, 6),
 (-8390990426696306640, 6),
 (-7711503493558257882, 6),
 (-7960547740899800424, 6),
 (6761302238723375284, 6),
 (6909425763893306957, 6),
 (3919631331559987259, 6),
 (6469532641009687095, 6),
 (-3762050624809075131, 6),
 (2457456989745858426, 6),
 (4672366138250246365, 6),
 (-115745122932672076, 6),
 (-8709469426135343491, 6),
 (-8632157328587157513, 6),
 (1568975333502569574, 5),
 (2438164847813111864, 5),
 (-7680158086903684464, 5),
 (8627960238260150357, 5),
 (6033948683327358476, 5),
 (-839574455

In [23]:
print(myrdd.count())
print(myrdd2.count())
print(myrdd3.count())
print(myrdd4.count())
print(myrdd5.count())
print(myrdd6.count())
print(myrdd7.count())
n_shingles = myrdd7.count()

20
20
40206
37936
40206
37936
37936


In [24]:
prime_modulo = nextprime(n_shingles)
prime_modulo

37951

In [25]:
def vectorize(shingles, shingles_all):
    #return shingles
    return [shingle in shingles for shingle in shingles_all]

myrdd7_list = myrdd7.collect()
myrdd8 = myrdd2.map(lambda x: vectorize(x, shingles_all=myrdd7_list))

In [26]:
myrdd8.take(2)

[[True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,


In [27]:
def compare_sets(vec, vec_all):
    ious = []
    vec_np = np.array(vec)
    
    for idx, vec_other in enumerate(vec_all):
        vec_other_np = np.array(vec_other)
        intersection = np.logical_and(vec_np, vec_other_np).sum()
        union = np.logical_or(vec_np, vec_other_np).sum()
        iou = intersection / union
        ious.append((idx, iou))
        
    return ious

In [28]:
vec_all = myrdd8.collect() 
myrdd9 = myrdd8.map(lambda x: compare_sets(x, vec_all=vec_all))

In [29]:
myrdd9_collected = myrdd9.collect()

In [30]:
myrdd9_collected[11]

[(0, 0.0019347037484885128),
 (1, 0.0011418783899514702),
 (2, 0.004890544946436889),
 (3, 0.0046443412368614035),
 (4, 0.010502625656414103),
 (5, 0.0),
 (6, 0.0018542555164101613),
 (7, 0.0025614754098360654),
 (8, 0.0067026624464717926),
 (9, 0.004962779156327543),
 (10, 0.0019553072625698325),
 (11, 1.0),
 (12, 0.002153316106804479),
 (13, 0.003216726980297547),
 (14, 0.003205128205128205),
 (15, 0.002745367192862045),
 (16, 0.005393478793822015),
 (17, 0.0027685492801771874),
 (18, 0.026589242053789732),
 (19, 0.0076937511728279225)]

In [31]:
df_data_collected[11]

Row(path='data/bbc/business/077.txt', topic='business', text='China had role in Yukos split-up China lent Russia $6bn (£3.2bn) to help the Russian government renationalise the key Yuganskneftegas unit of oil group Yukos, it has been revealed. The Kremlin said on Tuesday that the $6bn which Russian state bank VEB lent state-owned Rosneft to help buy Yugansk in turn came from Chinese banks. The revelation came as the Russian government said Rosneft had signed a long-term oil supply deal with China. The deal sees Rosneft receive $6bn in credits from China\'s CNPC. According to Russian newspaper Vedomosti, these credits would be used to pay off the loans Rosneft received to finance the purchase of Yugansk. Reports said CNPC had been offered 20% of Yugansk in return for providing finance but the company opted for a long-term oil supply deal instead. Analysts said one factor that might have influenced the Chinese decision was the possibility of litigation from Yukos, Yugansk\'s former owner,

In [32]:
df_data_collected[18]

Row(path='data/bbc/business/314.txt', topic='business', text='Yukos unit fetches $9bn at auction A little-known Russian company has bought the main production unit of oil giant Yukos at auction in Moscow. Baikal Finance Group outbid favourite Gazprom, the state-controlled gas monopoly, to buy Yuganskneftegas. Baikal paid 260.75bn roubles ($9.37bn: £4.8bn) for Yugansk - nowhere near the $27bn Russia says Yukos owes in taxes. Yukos reacted immediately by repeating its view that the auction was illegal in international and Russian law, and said Baikal had bought itself trouble. "The company considers that the victor of today\'s auction has bought itself a serious $9bn headache," said Yukos spokesman Alexander Shadrin. He said the company would continue to make "every lawful move" to protect tens of thousands of shareholders in Yukos from "this forcible and illegitimate removal of their property". Meanwhile, Tim Osborne, head of Yukos main shareholders\' group Menatep, said that Yukos may 

In [33]:
def minhash_func(x, param):
    # h(x) = (ax + b) % c
    a, b, c = param
    return (a * x + b) % c

def minhash_funcs(xs, params):
    signature = []
    
    for param in params:
        minhash_value_per_func = min([minhash_func(x, param) for x in xs])
        signature.append(minhash_value_per_func)
    
    return signature 

def minhash_param(c, seed):
    np.random.seed(seed)
    a = np.random.randint(1, c)
    np.random.seed(seed+123)
    b = np.random.randint(1, c)
    return a, b, c

def minhash_params(n, c, seed):
    np.random.seed(seed)
    seeds = np.random.randint(low=1, high=2*n, size=n)
    return [minhash_param(c, seeds[i]) for i in range(n)]

In [34]:
n_signatures = 100
params = minhash_params(n=n_signatures, c=prime_modulo, seed=32)
params[:3]

[(14101, 11074, 37951), (9700, 7047, 37951), (4763, 12816, 37951)]

In [35]:
myrdd10 = myrdd8.map(lambda x: list(np.argwhere(x).flatten()))

In [36]:
myrdd10_collected = myrdd10.collect()

In [37]:
#print(myrdd10_collected[2])
#print(len(myrdd10_collected[2]))

In [38]:
myrdd11 = myrdd10.map(lambda x: minhash_funcs(x, params))

In [39]:
myrdd11_collected = myrdd11.collect()

In [40]:
#myrdd11_collected[0]

In [41]:
len(myrdd11_collected[0])

100

In [42]:
def compare_signatures(vec, vec_all):
    similarities = []
    vec_np = np.array(vec)
    
    for idx, vec_other in enumerate(vec_all):
        vec_other_np = np.array(vec_other)
        
        similarities.append((idx, (vec_np == vec_other_np).sum() / vec_np.size))
        
    return similarities

In [43]:
vec_all_2 = myrdd11.collect() 
myrdd12 = myrdd11.map(lambda x: compare_signatures(x, vec_all=vec_all_2))

In [44]:
myrdd12_collected = myrdd12.collect()
myrdd12_collected[11]

[(0, 0.02),
 (1, 0.0),
 (2, 0.01),
 (3, 0.0),
 (4, 0.03),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.04),
 (9, 0.01),
 (10, 0.0),
 (11, 1.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.01),
 (17, 0.01),
 (18, 0.04),
 (19, 0.03)]

In [45]:
n_bands = 5
n_rows_per_band = int(n_signatures / n_bands)
print(n_rows_per_band)
n_buckets = 20
sim = (1/n_bands)**(1/n_rows_per_band)
print(sim)

20
0.9226808345905884


In [46]:
def lsh_hash(rows_in_band, n_buckets):
    return hash(tuple(rows_in_band)) % n_buckets

In [47]:
def lsh(vec, n_rows_per_band, n_bands, n_buckets):
    return [lsh_hash(rows_in_band=vec[n_rows_per_band*b : n_rows_per_band*(b + 1)], n_buckets=n_buckets) 
            for b in range(n_bands)] 

In [48]:
myrdd13 = myrdd11.map(lambda x: lsh(x, n_rows_per_band, n_bands, n_buckets))

In [49]:
myrdd13_collected = myrdd13.collect()
print(myrdd13_collected[11])
print(myrdd13_collected[18])

[14, 3, 15, 8, 18]
[12, 10, 15, 7, 8]


In [50]:
def candidates(vec, vec_all, hash_to_n_min=1):
    candidate_pairs = []
    vec_np = np.array(vec)
    
    for idx, vec_other in enumerate(vec_all):
        vec_other_np = np.array(vec_other)
        
        hash_to_n = (vec_np == vec_other_np).sum()
        
        if hash_to_n_min <= hash_to_n:
            candidate_pairs.append(idx)
        
    return candidate_pairs

In [51]:
vec_all_3 = myrdd13.collect()
myrdd14 = myrdd13.map(lambda x: candidates(vec=x, vec_all=vec_all_3)).zipWithIndex()

In [52]:
myrdd14_collected = myrdd14.collect()
print(myrdd14_collected[11])
print(myrdd14_collected[18])

([0, 3, 9, 11, 17, 18], 11)
([0, 6, 7, 11, 16, 18], 18)


In [53]:
myrdd14_collected

[([0, 4, 11, 13, 14, 17, 18], 0),
 ([1, 4, 8], 1),
 ([2, 5, 7, 16, 17], 2),
 ([3, 7, 11, 12, 17], 3),
 ([0, 1, 4, 14, 17], 4),
 ([2, 5, 12, 13, 14], 5),
 ([6, 9, 10, 15, 16, 18], 6),
 ([2, 3, 7, 18], 7),
 ([1, 8, 10, 12, 15, 19], 8),
 ([6, 9, 10, 11, 15, 19], 9),
 ([6, 8, 9, 10, 15, 19], 10),
 ([0, 3, 9, 11, 17, 18], 11),
 ([3, 5, 8, 12, 13], 12),
 ([0, 5, 12, 13, 19], 13),
 ([0, 4, 5, 14, 17], 14),
 ([6, 8, 9, 10, 15, 19], 15),
 ([2, 6, 16, 18], 16),
 ([0, 2, 3, 4, 11, 14, 17], 17),
 ([0, 6, 7, 11, 16, 18], 18),
 ([8, 9, 10, 13, 15, 19], 19)]

In [60]:
df_lsh = myrdd14.toDF(["lsh_candidates", "id"])

In [61]:
df_lsh.show()

+--------------------+---+
|      lsh_candidates| id|
+--------------------+---+
|[0, 4, 11, 13, 14...|  0|
|           [1, 4, 8]|  1|
|   [2, 5, 7, 16, 17]|  2|
|  [3, 7, 11, 12, 17]|  3|
|   [0, 1, 4, 14, 17]|  4|
|  [2, 5, 12, 13, 14]|  5|
|[6, 9, 10, 15, 16...|  6|
|       [2, 3, 7, 18]|  7|
|[1, 8, 10, 12, 15...|  8|
|[6, 9, 10, 11, 15...|  9|
|[6, 8, 9, 10, 15,...| 10|
|[0, 3, 9, 11, 17,...| 11|
|   [3, 5, 8, 12, 13]| 12|
|  [0, 5, 12, 13, 19]| 13|
|   [0, 4, 5, 14, 17]| 14|
|[6, 8, 9, 10, 15,...| 15|
|      [2, 6, 16, 18]| 16|
|[0, 2, 3, 4, 11, ...| 17|
|[0, 6, 7, 11, 16,...| 18|
|[8, 9, 10, 13, 15...| 19|
+--------------------+---+



In [62]:
df = \
    df_data.\
    join(df_lsh, df_data["id"] == df_lsh["id"], "outer").\
    drop(df_lsh.id).\
    sort("id", ascedning=True)
df.show()

+--------------------+--------+--------------------+---+--------------------+
|                path|   topic|                text| id|      lsh_candidates|
+--------------------+--------+--------------------+---+--------------------+
|data/bbc/business...|business|Criminal probe on...|  0|[0, 4, 11, 13, 14...|
|data/bbc/business...|business|'Post-Christmas l...|  1|           [1, 4, 8]|
|data/bbc/business...|business|Beijingers fume o...|  2|   [2, 5, 7, 16, 17]|
|data/bbc/business...|business|Jobs growth still...|  3|  [3, 7, 11, 12, 17]|
|data/bbc/business...|business|Winn-Dixie files ...|  4|   [0, 1, 4, 14, 17]|
|data/bbc/business...|business|US economy still ...|  5|  [2, 5, 12, 13, 14]|
|data/bbc/business...|business|Wall Street cheer...|  6|[6, 9, 10, 15, 16...|
|data/bbc/business...|business|China now top tra...|  7|       [2, 3, 7, 18]|
|data/bbc/business...|business|US to probe airli...|  8|[1, 8, 10, 12, 15...|
|data/bbc/business...|business|India unveils ant...|  9|[6, 9, 1

In [64]:
class Shingling():
    def __init__(self, k, df):
        # build k shingle map from corpus of docs
        pass
    
    def __call__(self, row):
        # get text from row and convert it to k-shingles
        pass
    
class CompareSets():
    def __init__(self,):
        pass
    
    def __call__(self, row):
        # compute Jaccard sim between two sets of shingles
        pass
    
class MinHashing():
    def __init__(self,):
        pass
    
    def __call__(self, row):
        pass

class CompareSignatures():
    def __init__(self,):
        pass
    
    def __call__(self, row):
        pass
    
class LSH():
    def __init__(self,):
        pass
    
    def __call__(self, row):
        pass